In this notebook we are going to see how to convert speech into text using Facebook Wav2Vec 2.0 model.Wav2Vec2 is a speech model that accepts a float array corresponding to the raw waveform of the speech signal. Wav2Vec2 model was trained using connectionist temporal classification (CTC) so the model output has to be decoded using Wav2Vec2Tokenizer.For learning more about it click on this [link](https://huggingface.co/transformers/model_doc/wav2vec2.html)

In [1]:
!pip install --upgrade transformers

     |████████████████████████████████| 7.2 MB 4.2 MB/s 
     |████████████████████████████████| 436 kB 39.9 MB/s 
     |████████████████████████████████| 268 kB 29.2 MB/s 
     |████████████████████████████████| 7.8 MB 40.1 MB/s 
     |████████████████████████████████| 53 kB 1.1 MB/s 
  Attempting uninstall: packaging
    Found existing installation: packaging 20.8
    Uninstalling packaging-20.8:
      Successfully uninstalled packaging-20.8
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.9.4
    Uninstalling tokenizers-0.9.4:
      Successfully uninstalled tokenizers-0.9.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.2.2
    Uninstalling transformers-4.2.2:
      Successfully uninstalled transformers-4.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-git 0.11.0 requires nb

In [2]:
import transformers
print(transformers.__version__)

4.30.2


If you don't see at least 4.3.0 version,then upgrade it

In [3]:
!pip install --upgrade torch

     |████████████████████████████████| 887.5 MB 4.0 kB/s 
     |████████████████████████████████| 21.0 MB 7.6 MB/s 
     |████████████████████████████████| 317.1 MB 21 kB/s 
     |████████████████████████████████| 849 kB 5.9 MB/s 
     |████████████████████████████████| 557.1 MB 6.9 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.7.0
    Uninstalling torch-1.7.0:
      Successfully uninstalled torch-1.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.2.5 requires torch<1.8,>=1.7.0, but you have torch 1.13.1 which is incompatible.
allennlp 2.0.1 requires torch<1.8.0,>=1.6.0, but you have torch 1.13.1 which is incompatible.
allennlp 2.0.1 requires transformers<4.3,>=4.1, but you have transformers 4.30.2 which is incompatible.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' comm

### Import Libraries

In [4]:
import librosa
import torch
import IPython.display as display
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
import numpy as np

### Load pre-trained Wav2Vec model

In [5]:
#load pre-trained model and tokenizer
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/opt/conda/lib/python3.7/site-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:795: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  FutureWarning,


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Load Audio file

In [6]:
#load audio file 
audio, sampling_rate = librosa.load("/kaggle/input/audio-data/audio1.m4a",sr=16000)

/opt/conda/lib/python3.7/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [7]:
audio,sampling_rate

(array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 16000)

# Play the Audio

In [8]:
# audio
display.Audio("/kaggle/input/audio-data/audio1.m4a", autoplay=True)

### Speech to Text

First of all tokenize the input values,take the maximum prediction from the logit and then extraxt the text

In [9]:
input_values = tokenizer(audio, return_tensors = 'pt').input_values
input_values

tensor([[-0.0002, -0.0002, -0.0002,  ..., -0.0002, -0.0002, -0.0002]])

In [10]:
# store logits (non-normalized predictions)
logits = model(input_values).logits
logits

tensor([[[ 14.7457, -27.0516, -26.7098,  ...,  -5.9108,  -6.2960,  -7.6940],
         [ 14.8289, -27.1641, -26.8184,  ...,  -5.9686,  -6.3497,  -7.7359],
         [ 14.8723, -27.1996, -26.8532,  ...,  -5.9437,  -6.3394,  -7.7483],
         ...,
         [ 14.6778, -27.3817, -27.0478,  ...,  -6.5107,  -7.2095,  -7.6820],
         [ 14.6694, -27.5049, -27.1693,  ...,  -6.5099,  -7.2421,  -7.3955],
         [ 14.5684, -27.3072, -26.9728,  ...,  -6.5685,  -7.2532,  -7.7025]]],
       grad_fn=<ViewBackward0>)

In [11]:
# store predicted id's
# pass the logit values to softmax to get the predicted values
predicted_ids = torch.argmax(logits, dim =-1)

In [12]:
# pass the prediction to the tokenzer decode to get the transcription
transcriptions = tokenizer.decode(predicted_ids[0])

In [13]:
transcriptions

'HER LOGAGE HOW ARE YOU HOW ARE YOU DOING BRO I AM GOING FINE HARTOARTOU HERLORD'